Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and few-shot settings. Select one of the tasks:

    QA of simple questions (capital cities, currency, president...)
    Classification (sentiment, news topic, language identification...)
    Extracting facts from the given text (names, numbers, countries...)

Design a prompt for the task and test a model in zero-shot, one-shot, and few-shot settings. We suggest you try smaller models gpt-neo-1.3B, or gpt-neo-2.7B but you can try out any model suitable for text generation.  To load a medium-sized model on Colab, you can use e.g.

##LOGIN

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install --quiet transformers accelerate bitsandbytes>0.37.0

In [3]:
!pip install -U bitsandbytes

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch

In [5]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
)


In [6]:
# 1: Load the model and tokenizer
G_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", device_map="auto", quantization_config=quantization_config)
G_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

### OPTIONS

In [7]:
pipeline = pipeline(
    "text-generation",
    model=G_model,
    tokenizer=G_tokenizer,
    device_map="auto",
)



## QUESTION ANSWERING

In [14]:
questions = [
    "What is the currency of Finland?",
    "What are five first prime numbers?",
    "What is the circumfence of earth?",
    "Is santa claus real?",
]

for q in questions:
    messages = [
    {"role": "system", "content": "You are now capable of answering simple questions"},
    {"role": "user", "content": f"Can you tell me answer to thiss: {q}"}
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=50,
    )

    print("", end="\n\n")
    print(outputs[0]["generated_text"][-1]["content"])
    print("", end="\n\n")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




The currency of Finland is the Euro.




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




The first five prime numbers are:

1. 2
2. 3
3. 5
4. 7
5. 11




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




The Earth's circumference can be calculated in two ways: equatorial and polar.

The equatorial circumference of the Earth is approximately 40,075 kilometers (24,901 miles).

The polar circumference of the Earth is approximately 40,008 kilometers (




The answer to this question can vary depending on who you ask. 

For many children, Santa Claus is a real and magical figure who brings them gifts on Christmas Eve. 

However, for adults, the idea of Santa Claus is often seen as a




Good enough...

## LANGUAGE IDENTIFICATION


### ZERO-SHOT


In [9]:
examples = ["hejsan kille", "moi mitä kuuluu", "ni hao"]

for e in examples:
  messages = [
    {"role": "system", "content": "You are now language expert"},
    {"role": "user", "content": f"Can you tell what language is this: {e}, answer with the language only."}
    ]

  outputs = pipeline(
      messages,
      max_new_tokens=50,
  )

  print("", end="\n\n")
  print(outputs[0]["generated_text"][-1]["content"])
  print("", end="\n\n")





Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Swedish




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Finnish.




Mandarin Chinese




### ONE-SHOT

In [10]:
actual = ["Por favor", "Je ne comprends pas", "Би хоол хийдэг "] # spanish, french, mongolian
example_pairs = [("hejsan kille", "Swedish"), ("moi mitä kuuluu", "Finnish"), ("ni hao", "Mandarin")]




for i,a in enumerate(actual):
  messages = [
    {"role": "system", "content": "You are now language expert"},
    {"role": "user", "content": f"For example {example_pairs[i][0]} is {example_pairs[i][1]}"},
    {"role": "user", "content": f"Can you tell what language is this: {a}, answer with the language only."}
    ]

  outputs = pipeline(
      messages,
      max_new_tokens=50,
  )

  print("", end="\n\n")
  print(outputs[0]["generated_text"][-1]["content"])
  print("", end="\n\n")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Spanish




You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




French




Mongolian




### FEW-SHOT

No point making few shot as it is 6/6 already... Let's try something different.

## FACT EXTRACTION

In [11]:
text = '''
Penalties
Main article: Penalty (ice hockey)
An ice hockey player enters the penalty box. Players may be sent to the penalty box for rule infractions, forcing their team to play with one less player for a specified time.
In ice hockey, infractions of the rules lead to a play stoppage whereby the play is restarted at a faceoff. Some infractions result in a penalty on a player or team. In the simplest case, the offending player is sent to the penalty box and their team must play with one less player on the ice for a designated time. Minor penalties last for two minutes, major penalties last for five minutes, and a double minor penalty is two consecutive penalties of two minutes duration. A single minor penalty may be extended by two minutes for causing visible injury to the victimized player. This is usually when blood is drawn during high sticking. Players may be also assessed personal extended penalties or game expulsions for misconduct in addition to the penalty or penalties their team must serve. The team that has been given a penalty is said to be playing short-handed while the opposing team is on a power play.
A two-minute minor penalty is often charged for lesser infractions such as tripping, elbowing, roughing, high-sticking, delay of the game, too many players on the ice, boarding, illegal equipment, charging (leaping into an opponent or body-checking him after taking more than two strides), holding, holding the stick (grabbing an opponent's stick), interference, hooking, slashing, kneeing, unsportsmanlike conduct (arguing a penalty call with referee, extremely vulgar or inappropriate verbal comments), "butt-ending" (striking an opponent with the knob of the stick), "spearing" (jabbing an opponent with the blade of the stick), or cross-checking. As of the 2005–2006 season, a minor penalty is also assessed for diving, where a player embellishes or simulates an offence. More egregious fouls may be penalized by a four-minute double-minor penalty, particularly those that injure the victimized player. These penalties end either when the time runs out or when the other team scores during the power play. In the case of a goal scored during the first two minutes of a double-minor, the penalty clock is set down to two minutes upon a score, effectively expiring the first minor penalty.
A skater cross-checking his opponent, checking him with the shaft of his stick with two hands.
A skater hooking his opponent, using his stick to restrain him.
These are examples of rule infractions in the sport; a penalty may be assessed against the players committing them.
Five-minute major penalties are called for especially violent instances of most minor infractions that result in intentional injury to an opponent, or when a minor penalty results in visible injury (such as bleeding), as well as for fighting. Major penalties are always served in full; they do not terminate on a goal scored by the other team. Major penalties assessed for fighting are typically offsetting, meaning neither team is short-handed and the players exit the penalty box upon a stoppage of play following the expiration of their respective penalties. The foul of boarding (defined as "check[ing] an opponent in such a manner that causes the opponent to be thrown violently in the boards")[8] is penalized either by a minor or major penalty at the discretion of the referee, based on the violent state of the hit. A minor or major penalty for boarding is often assessed when a player checks an opponent from behind and into the boards.
Some varieties of penalty do not require the offending team to play a man short. Concurrent five-minute major penalties in the NHL usually result from fighting. In the case of two players being assessed five-minute fighting majors, both the players serve five minutes without their team incurring a loss of player (both teams still have a full complement of players on the ice). This differs with two players from opposing sides getting minor penalties, at the same time or at any intersecting moment, resulting from more common infractions. In this case, both teams will have only four skating players (not counting the goaltender) until one or both penalties expire (if one penalty expires before the other, the opposing team gets a power play for the remainder of the time); this applies regardless of current pending penalties. In the NHL, a team always has at least three skaters on the ice. Thus, ten-minute misconduct penalties are served in full by the penalized player, but his team may immediately substitute another player on the ice unless a minor or major penalty is assessed in conjunction with the misconduct (a two-and-ten or five-and-ten). In this case, the team designates another player to serve the minor or major; both players go to the penalty box, but only the designee may not be replaced, and he is released upon the expiration of the two or five minutes, at which point the ten-minute misconduct begins. In addition, game misconducts are assessed for deliberate intent to inflict severe injury on an opponent (at the officials' discretion), or for a major penalty for a stick infraction or repeated major penalties. The offending player is ejected from the game and must immediately leave the playing surface (he does not sit in the penalty box); meanwhile, if an additional minor or major penalty is assessed, a designated player must serve out of that segment of the penalty in the box (similar to the above-mentioned "two-and-ten"). In some rare cases, a player may receive up to nineteen minutes in penalties for one string of plays. This could involve receiving a four-minute double-minor penalty, getting in a fight with an opposing player who retaliates, and then receiving a game misconduct after the fight. In this case, the player is ejected and two teammates must serve the double-minor and major penalties.
A skater taking a penalty shot, with a referee in the background. A referee may award a player with a penalty shot if they assess an infraction stopped the player from a clear scoring opportunity.
A penalty shot is awarded to a player when the illegal actions of another player stop a clear scoring opportunity, most commonly when the player is on a breakaway. A penalty shot allows the obstructed player to pick up the puck on the centre red-line and attempt to score on the goalie with no other players on the ice, to compensate for the earlier missed scoring opportunity. A penalty shot is also awarded for a defender other than the goaltender covering the puck in the goal crease, a goaltender intentionally displacing his own goal posts during a breakaway to avoid a goal, a defender intentionally displacing his own goal posts when there is less than two minutes to play in regulation time or at any point during overtime, or a player or coach intentionally throwing a stick or other object at the puck or the puck carrier and the throwing action disrupts a shot or pass play.
Officials also stop play for puck movement violations, such as using one's hands to pass the puck in the offensive end, but no players are penalized for these offences. The sole exceptions are deliberately falling on or gathering the puck to the body, carrying the puck in the hand, and shooting the puck out of play in one's defensive zone (all penalized two minutes for delay of game).
In the NHL, a unique penalty applies to the goalies. The goalies now are forbidden to play the puck in the "corners" of the rink near their own net. This will result in a two-minute penalty against the goalie's team. Only in the area in front of the goal line and immediately behind the net (marked by two red lines on either side of the net) can the goalie play the puck.
An additional rule that has never been a penalty, but was an infraction in the NHL before recent rules changes, is the two-line offside pass. Prior to the 2005–06 NHL season, play was stopped when a pass from inside a team's defending zone crossed the centre line, with a face-off held in the defending zone of the offending team. Now, the centre line is no longer used in the NHL to determine a two-line pass infraction, a change that the IIHF had adopted in 1998. Players are now able to pass to teammates who are more than the blue and centre ice red line away.
The NHL has taken steps to speed up the game of hockey and create a game of finesse, by reducing the number of illegal hits, fights, and "clutching and grabbing" that occurred in the past. Rules are now more strictly enforced, resulting in more penalties, which provides more protection to the players and facilitates more goals being scored. The governing body for United States' amateur hockey has implemented many new rules to reduce the number of stick-on-body occurrences, as well as other detrimental and illegal facets of the game ("zero tolerance").
In men's hockey, but not in women's, a player may use his hip or shoulder to hit another player if the player has the puck or is the last to have touched it. This use of the hip and shoulder is called body checking. Not all physical contact is legal—in particular, hits from behind, hits to the head and most types of forceful stick-on-body contact are illegal.
A referee calls a delayed penalty, which sees play continue until a goal is scored, or the opposing team regains control of the puck.
A delayed penalty call occurs when an offence is committed by the team that does not have possession of the puck. In this circumstance the team with possession of the puck is allowed to complete the play; that is, play continues until a goal is scored, a player on the opposing team gains control of the puck, or the team in possession commits an infraction or penalty of their own. Because the team on which the penalty was called cannot control the puck without stopping play, it is impossible for them to score a goal. In these cases, the team in possession of the puck can pull the goalie for an extra attacker without fear of being scored on. It is possible for the controlling team to mishandle the puck into their own net. If a delayed penalty is signalled and the team in possession scores, the penalty is still assessed to the offending player, but not served. In 2012, this rule was changed by the United States' National Collegiate Athletic Association (NCAA) for college level hockey. In college games, the penalty is still enforced even if the team in possession scores.[9]
'''

In [12]:
messages = [
  {"role": "system", "content": "You are now expert in ice-hockey rules and fact extraction from text"},
  {"role": "user", "content": "Could you please extract all different ice-hockey penalties from the following text"},
  {"role":"user","content": f"{text}"}
  ]

outputs = pipeline(
    messages,
    max_new_tokens=500,
)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [13]:
#only the output of model
print(outputs[0]["generated_text"][-1]["content"])

Here are the different ice-hockey penalties extracted from the given text:

1. **Minor penalty**: 2 minutes, for lesser infractions such as:
	* Tripping
	* Elbowing
	* Roughing
	* High-sticking
	* Delay of the game
	* Too many players on the ice
	* Boarding
	* Illegal equipment
	* Charging
	* Holding
	* Holding the stick
	* Interference
	* Hooking
	* Slashing
	* Kneeing
	* Unsportsmanlike conduct
	* "Butt-ending"
	* "Spearing"
	* Cross-checking
	* Diving
2. **Double-minor penalty**: 4 minutes, for more egregious fouls such as:
	* Intentional injury to an opponent
	* Visible injury (such as bleeding)
	* Fighting
3. **Major penalty**: 5 minutes, for especially violent instances of most minor infractions that result in intentional injury to an opponent, or when a minor penalty results in visible injury.
4. **Concurrent major penalties**: 5 minutes, for two players from opposing sides getting minor penalties at the same time or at any intersecting moment, resulting from more common infract

## SUMMARY

I wanted to try this model exactly because it was one of the two to be used in the course project.

Findings:

* No need to use few-shot for those seemingly trivial language detection tasks
* Found out the facts from the wikipedia page also, i tried to look for error but could not find any

Additional findings, altho i was using 8bit quantization it still hogged up 11GB of GPU RAM, so if i would like to do more runs, it would need to be 4bit.
